In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

import sys
from types import SimpleNamespace

import numpy as np

sys.path.insert(0, "/home/jovyan/pypulseq/src")
import pypulseq as pp

sys.path.insert(0, "/home/jovyan/qrage/src")
from qrage.sequence.qrage import QRAGE

In [ ]:
seq_write = False
seq_debug = True
seq_plot = True
seq_calculate_gradient_spectrum = True
seq_check_timing = True
seq_test_report = True
seq_filename = "/home/jovyan/qrage/seq/qrage.seq"

In [ ]:
seq = pp.Sequence()

system = pp.Opts(
    max_grad=28,
    grad_unit="mT/m",
    max_slew=200,
    slew_unit="T/m/s",
    rf_ringdown_time=40e-6,
    rf_dead_time=100e-6,
    # adc_dead_time=0,
    adc_dead_time=10e-6,
)

if seq_debug:
    num_spokes = 2
    num_sets = 19
    num_echoes = 9
    num_partitions_per_block = 16
    num_autocalibration_lines = 0
    acceleration_factor = 1
    fov = np.array([256, 256, 16]) * 1e-3
    matrix_size = np.array([256, 256, 16])
    readout_bandwidth = 390.625
else:
    num_spokes = 8
    num_sets = 19
    num_echoes = 9
    num_partitions_per_block = 16
    num_autocalibration_lines = 32
    acceleration_factor = 2
    fov = np.array([256, 256, 160]) * 1e-3
    matrix_size = np.array([256, 256, 160])
    readout_bandwidth = 390.625

axes = SimpleNamespace()

xyz = ["x", "y", "z"]

axes.d1 = "x"  # Readout dimension
axes.d2 = "y"  # Inner phase-encoding loop
axes.d3 = "z"  # Outer phase-enconding loop

axes.n1 = xyz.index(axes.d1)
axes.n2 = xyz.index(axes.d2)
axes.n3 = xyz.index(axes.d3)

In [ ]:
seq.set_definition("FOV", fov.tolist())
seq.set_definition("RES", matrix_size.tolist())
seq.set_definition("Name", "QRAGE")

In [ ]:
qrage = QRAGE(
    fov,
    matrix_size,
    axes,
    readout_bandwidth,
    num_spokes,
    num_sets,
    num_echoes,
    num_partitions_per_block,
    num_autocalibration_lines,
    acceleration_factor,
    adiabatic_pulse_type="hypsec_n",
    adiabatic_pulse_overdrive=2.0,
    debug=False,
    system=system,
)

In [ ]:
qrage.run(seq)

In [ ]:
qrage.get_timing(seq)
print(
    "TR %s ms" % np.round(qrage.TR, decimals=1),
    "dTI %s ms" % np.round(qrage.dTI, decimals=1),
    "TI0 %s ms" % np.round(qrage.TI0, decimals=1),
    "dTE %s ms" % np.round(qrage.dTE, decimals=1),
    "TE0 %s ms" % np.round(qrage.TE0, decimals=1),
)

In [ ]:
if seq_plot:
    seq.plot(time_range=[0, 0.1], grad_disp="mT/m", time_disp="ms", show_blocks=False)

In [ ]:
if seq_calculate_gradient_spectrum:
    spects, spects_sos, freq, _ = seq.calculate_gradient_spectrum()
    res_freqs = freq[np.argmax(spects, axis=1)]
    print("Resonance frequencies of gradients (x, y, z) are: ", res_freqs)

In [ ]:
if seq_check_timing:
    ok, error_report = seq.check_timing()
    if ok:
        print("Timing check passed")
    else:
        print("Timing check failed")
        [print(e) for e in error_report]

In [ ]:
if seq_test_report:
    rep = seq.test_report()
    print(rep)

In [ ]:
if seq_write:
    seq.write(seq_filename)